In [26]:
import IPython
import matplotlib.pyplot as plt
from matplotlib.backend_bases import RendererBase
from scipy import signal
from scipy.io import wavfile
#import soundfile as sf
import os
import numpy as np
from PIL import Image
from scipy.fftpack import fft

%matplotlib inline

In [27]:
def readAudios(path):    
    samplerate, data = wavfile.read(path)
    return samplerate, data

In [28]:
def log_specgram(data, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    #t Array of segment times.
    freqs,t, spec = signal.spectrogram(data,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, np.log(spec.T.astype(np.float32) + eps)

In [223]:
#Retorna array 2 filas y num_columns*num_rows en la primera columna esta la media del bloque y en la segunda la std
def split_blocks(num_rows, num_columns, spectogram):
    features = np.zeros((2,num_columns*num_rows))
    featuresColumns = np.split(spectogram,[num_columns])
    featuresColumns = np.array_split(x, num_columns,1)
    cont = 0
    for i in range(len(featuresColumns)):
        featureRow= np.array_split(featuresColumns[i], num_rows,0) 
        for j in range(len(featureRow)):
            features[0][cont]=(np.mean(featureRow[j]))
            features[1][cont]=(np.std(featureRow[j]))
            cont = cont +1
    return features #fila con el numero carácteristicas con la media y std

In [224]:
def get_features_audio(path, num_rows, num_columns):
    sample_rate, data = readAudios(path)
    freqs, spectogram = log_specgram(data, sample_rate)
    features = split_blocks(num_rows, num_columns, spectogram)
    return features
    

In [136]:
tracks = ["./data/five/0ab3b47d_nohash_2.wav"]
for track in tracks:
    sample_rate, data = readAudios(track)
    freqs, spectogram = log_specgram(data, sample_rate)
    print("Tracks", spectogram.shape)

Tracks (97, 161)


In [239]:
PATHTEST = 'data/five/0ab3b47d_nohash_2.wav'
features = get_features_audio(PATHTEST, 9, 8)

print(features)

[[0.40354974 0.3240469  0.72919817 0.67114547 0.55066568 0.69695338
  0.46587599 0.27403153 0.19659035 0.47712884 0.9155578  0.44831144
  0.2638911  0.52586737 0.5101367  0.38025404 0.49171471 0.10852465
  0.72211618 0.25329702 0.83983018 0.47526119 0.56625264 0.31339627
  0.15170186 0.69107206 0.48470841 0.67965446 0.41770556 0.32420602
  0.28757273 0.51376351 0.20485005 0.26938964 0.67734924 0.11706267
  0.61818135 0.82906777 0.18022102 0.77951871 0.54400098 0.05925482
  0.89120741 0.80170043 0.59166326 0.79449814 0.7872091  0.86706551
  0.28008935 0.86995975 0.54716857 0.74805187 0.50024709 0.16982355
  0.70763651 0.79423388 0.4891853  0.78844746 0.36500779 0.50680583
  0.8909416  0.12765875 0.58692624 0.81536596 0.13495449 0.83850628
  0.4374018  0.0215857  0.13278166 0.07704332 0.45920548 0.32315971]
 [0.23612981 0.19418122 0.03778567 0.09309314 0.36236451 0.28882646
  0.0292934  0.11160466 0.06293783 0.30475996 0.07622606 0.3685598
  0.21027559 0.05163382 0.43990708 0.31411408 0.